In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output

In [2]:
sub_path = "D:\dataset\submit\kaggle_iceberg\stacking"
all_files = os.listdir(sub_path)
print all_files

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

['sub_200_ens_densenet.csv', 'sub_blend009.csv', 'sub_fcn.csv', 'sub_keras_beginner.csv', 'sub_TF_keras.csv']


,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
0,5941774d,1.943216e-02,0.094676,0.008171,0.010215,0.005586
1,4023181e,3.168809e-02,0.952222,0.638348,0.237662,0.145927
2,b20200e4,4.000000e-08,0.167771,0.008061,1.000000,0.000015
3,e7f018bb,9.925741e-01,0.989356,0.999506,0.999533,0.999914
4,4371c8c3,2.215107e-02,0.900321,0.774739,0.994269,0.033843


In [3]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
is_iceberg_0,1.000000,0.420791,0.809320,0.516280,0.890434
is_iceberg_1,0.420791,1.000000,0.489134,0.432111,0.493308
is_iceberg_2,0.809320,0.489134,1.000000,0.490128,0.789509
is_iceberg_3,0.516280,0.432111,0.490128,1.000000,0.548663
is_iceberg_4,0.890434,0.493308,0.789509,0.548663,1.000000


In [4]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [5]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.7
cutoff_hi = 0.3

In [6]:
# load the model with best base performance
sub_base = pd.read_csv('D:\dataset\submit\kaggle_iceberg\stacking\sub_200_ens_densenet.csv')

In [7]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))

concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')